<style type="text/css">
.demo-info {
    font-style: italic;
}
</style>

<div class="demo-info">

Create deep links

Below is a widget for creating deep links to start collecting patient data with CommonHealth Cloud

Choices to make:

- Select the Partner application the data will be shared with.

  We currently have two partner applications for collecting data: 'testing' for testing, and the Pre-MVP itself.
  Use 'testing' if you are still working things out, like testing this widget page, or 'Pre-MVP' if you want to create a deep link that will add data to production Pre-MVP demos.
- Select the patient id. This is the id you will use to fetch the patient's uploaded data in the future.
  In a 'real' deployment, this will be set by the hea

In a 'real' deployment, the above choices will be made for you.

Additional choices:

- Select a subset of data types to request (default: heart rate and blood pressure)

</div>

In [ ]:
import html
import secrets
from textwrap import dedent

import ipywidgets as W
from IPython.display import HTML, display
from jupyter_health import JupyterHealthCHClient

ch_clients = {key: JupyterHealthCHClient(key) for key in ("testing", "prod")}

available_scopes = (
    "OMHealthResource.HeartRate.Read",
    "OMHealthResource.BloodPressure.Read",
)
_width = "400px"

import jwt

demo_link = jwt.encode({"demo": secrets.token_urlsafe(500)}, b"asdf")


@W.interact(
    partner=W.Dropdown(
        options={
            ch_client.get_configuration()["name"]: key
            for key, ch_client in ch_clients.items()
        },
        value="testing",
        layout=W.Layout(width=_width),
    ),
    patient_id="",
    # expires_days=W.IntSlider(value=30, min=1, max=30),
    blood_pressure=W.Checkbox(value=True, description="Blood pressure"),
    heart_rate=W.Checkbox(value=False, description="Heart rate"),
).options(manual=True, manual_name="Invite patient")
def create_deep_link(partner, patient_id, blood_pressure, heart_rate):
    """Create a deep link with inputs"""
    if not patient_id:
        print("patient id is required")
        return

    scopes = []
    if blood_pressure:
        scopes.append("OMHealthResource.BloodPressure.Read")
    if heart_rate:
        scopes.append("OMHealthResource.HeartRate.Read")
    if not scopes:
        print("Must specify at least one type of data to collect.")
        return
        
    ch_client = ch_clients[partner]
    if patient_id == "test":
        deeplink = "https://appdev.tcpdev.org/m/phr/cloud-sharing/authorize?authorization_request=" + demo_link
    else:
        deeplink = ch_client.construct_authorization_request_deeplink(
            patient_id,
            scope=" ".join(scopes),
            expiration_seconds=30 * (24 * 3600),
        )
    display(
        HTML(
            dedent(
                f"""
    <style type="text/css">
    .deep-link-box {{
      border: 2px solid rgba(84, 174, 255, 0.4);
      padding: 16px;
      border-radius: 8px;
    }}
    .deep-link-box pre {{
      border: 2px solid rgba(84, 174, 255, 0.4);
      background-color: white;
      padding: 16px;
      border-radius: 8px;
    }}
    .deep-link-box button {{
      width: 100%;
      font-size: 150%;
      margin-top: 8px;
      padding: 8px;
      height: auto;
    }}
    </style>
    <i>
      demo: select message below and email to Simona.
      In a real situation, this step would be automated in MyChart.
    </i>
  <br></br>
    <div class="deep-link-box">
      <p><i>Subject:</i> Share your blood pressure data with UCSF</p>
      <p>
        Dear Simona, 
      </p>
      <p>
        Your care team at UCSF invites you to share data from your iHealth Blood Pressure Cuff through CommonHealth. 
        Please accept the invitation by clicking on the link below.
      </p>
      <a href="{html.escape(deeplink)}">Click here to start uploading data from CommonHealth</a>
    </div>
    """
            )
        )
    )


create_deep_link.widget.children[-2].button_style = "primary"
create_deep_link.widget.layout.border = "4px solid #cff"
create_deep_link.widget.layout.padding = "16px"
create_deep_link.widget.layout.margin = "16px"